In [1]:
from dolfinx import mesh as msh, fem, io, default_scalar_type
from mpi4py import MPI
import os
import torch
import numpy as np
import ufl
from utils.transforms import create_cell_ind_to_grid


def mesh_to_x(mesh, cell_ind_to_grid, epsK, bK, cK, fK, non_perturbed=False, device=None, dtype=None):
    lst = []
    for v in mesh.geometry.x[:,0:2][mesh.geometry.dofmap]:
        xK = np.concat((v, epsK, bK, cK, fK), axis=None)
        lst.append(xK)
        
    xnp=np.array(lst).T
    x = torch.tensor(xnp[:,cell_ind_to_grid], requires_grad=False, device=device, dtype=dtype)

    x = torch.nn.functional.pad(x, pad=(1,1,1,1))

    nx, ny  = cell_ind_to_grid.shape
    # left column: set v0, v1 to v0, v1 of neighboring column
    x[0:4,1:ny+1, 0] = x[0:4,1:ny+1, 1]
    # set collapsed vertices v2 and v3 to v1 at the same position
    x[4:6,1:ny+1, 0] = x[2:3,1:ny+1, 0]
    x[6:8,1:ny+1, 0] = x[2:3,1:ny+1, 0]


    # right column: set v0 to v2 of neighboring column
    x[0:4,1:ny+1, -1] = x[4:8,1:ny+1, -1 -1]
    # set collapsed vertices v2 and v3 to v1 at the same position
    x[4:6,1:ny+1, -1] = x[2:3,1:ny+1, -1]
    x[6:8,1:ny+1, -1] = x[2:3,1:ny+1, -1]


    # top row: set v0 to v0 of neighboring row
    x[0:2,0,1:nx+1] = x[2:4,1,1:nx+1]
    # top row: set v1 to v3 of neighboring row
    x[2:4,0,1:nx+1] = x[6:8,1,1:nx+1]
    # set collapsed vertices v2 and v3 to v1 at the same position
    x[4:6,0,1:nx+1] = x[2:3,0,1:nx+1]
    x[6:8,0,1:nx+1] = x[2:3,0,1:nx+1]

    # bottom row: set v0 to v0 of neighboring row
    x[0:2,-1,1:nx+1] = x[0:2,-1-1,1:nx+1]
    # bottom row: set v1 to v2 of neighboring row
    x[2:4,-1,1:nx+1] = x[4:6,-1-1,1:nx+1]
    # set collapsed vertices v2 and v3 to v1 at the same position
    x[4:6,-1,1:nx+1] = x[2:3,-1,1:nx+1]
    x[6:8,-1,1:nx+1] = x[2:3,-1,1:nx+1]

    # set upper left corner to v0 of neigboring horizontal edge
    x[0:2,0,0] = x[0:2,0,1]
    # set remaining vertices to v0
    x[2:4,0,0] = x[0:2,0,0]
    x[4:6,0,0] = x[0:2,0,0]
    x[6:8,0,0] = x[0:2,0,0]

    # set bottom left corner to v0 of neighboring horizontal edge
    x[0:2,-1,0] = x[0:2,-1,1]
    # set remaining vertices to v0
    x[2:4,-1,0] = x[0:2,-1,0]
    x[4:6,-1,0] = x[0:2,-1,0]
    x[6:8,-1,0] = x[0:2,-1,0]

    # set upper right corner to v1 of neighboring horizontal edge
    x[0:2,0,-1] = x[2:4,0,-1-1]
    # set remaining vertices to v0
    x[2:4,0,-1] = x[0:2,0,-1]
    x[4:6,0,-1] = x[0:2,0,-1]
    x[6:8,0,-1] = x[0:2,0,-1]

    # set bottom right corner to v1 of neighboring horizontal edge
    x[0:2,-1,-1] = x[2:4,-1,-1-1]
    # set remaining vertices to v0
    x[2:4,-1,-1] = x[0:2,-1,-1]
    x[4:6,-1,-1] = x[0:2,-1,-1]
    x[6:8,-1,-1] = x[0:2,-1,-1]

    # set the dirichlet boundary condition
    x[11,0,:] = 1
    x[11,-1,:] = 1
    x[11,:,0] = 1
    x[11,:,-1] = 1

    if non_perturbed:
        if np.all(bK == np.array([1,0])):
            x[12,0,:] = (x[0,0,:] + x[2,0,:])/2
            x[12,-1,:] = (x[0,-1,:] + x[2,-1,:])/2
            x[12,:,0] = (x[0,:,0] + x[2,:,0])/2
            x[12,:,-1] = (x[0,:,-1] + x[2,:,-1])/2
        if np.all(bK == np.array([-1,0])):
            x[12,0,:] = ((x[0,0,:] + x[2,0,:])/2).flip(0)
            x[12,-1,:] = ((x[0,-1,:] + x[2,-1,:])/2).flip(0)
            x[12,:,0] = ((x[0,:,0] + x[2,:,0])/2).flip(0)
            x[12,:,-1] = ((x[0,:,-1] + x[2,:,-1])/2).flip(0)
        if np.all(bK == np.array([0,1])):
            x[12,0,:] = (x[1,0,:] + x[3,0,:])/2
            x[12,-1,:] = (x[1,-1,:] + x[3,-1,:])/2
            x[12,:,0] = (x[1,:,0] + x[3,:,0])/2
            x[12,:,-1] = (x[1,:,-1] + x[3,:,-1])/2
        if np.all(bK == np.array([0,-1])):
            x[12,0,:] = ((x[1,0,:] + x[3,0,:])/2).flip(0)
            x[12,-1,:] = ((x[1,-1,:] + x[3,-1,:])/2).flip(0)
            x[12,:,0] = ((x[1,:,0] + x[3,:,0])/2).flip(0)
            x[12,:,-1] = ((x[1,:,-1] + x[3,:,-1])/2).flip(0)


    return x

In [1]:
from dolfinx import mesh as msh
from mpi4py import MPI
from dolfinx import fem
from dolfinx import default_scalar_type
import ufl

def pde_data(domain):
    Wh = fem.functionspace(domain, ('P', 2))
    domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)
    boundary_facets = msh.exterior_facet_indices(domain.topology)
    boundary_dofs = fem.locate_dofs_topological(Wh, domain.topology.dim-1, boundary_facets)
    eps = fem.Constant(domain, default_scalar_type(1e-8))
    b = ufl.as_vector((fem.Constant(domain, default_scalar_type(1.0)),fem.Constant(domain, default_scalar_type(0.0))))
    c = fem.Constant(domain, default_scalar_type(0.0))
    f = fem.Constant(domain, default_scalar_type(1.0))
    x = ufl.SpatialCoordinate(domain)
    expr = x[0]
    u_exact = fem.Expression(expr, Wh.element.interpolation_points())
    uD = fem.Function(Wh)
    uD.interpolate(u_exact)
    bcs = [fem.dirichletbc(uD, boundary_dofs)]
    return Wh,eps,b,c,f,bcs,None

In [ ]:
import pyvista as pv
from supg import supg
from dolfinx import mesh as msh
from mpi4py import MPI

nx , ny = 5,5
domain = msh.create_unit_square(MPI.COMM_WORLD, nx, ny, msh.CellType.triangle)

sd = supg.data(domain=domain, pde_data=pde_data, boundary_eval=False)
sd.set_weights(1e-2)
stg = sd.create_stage()

weights = sd.yh.x.array


p = pv.Plotter()


stg.add_data(sd.uh)
p.add_mesh(stg.grid, show_edges=True)

p.show()

Widget(value='<iframe src="http://localhost:52434/index.html?ui=P_0x1738cd2b0_0&reconnect=auto" class="pyvista…

In [65]:
import pyvista as pv
from supg import supg
from dolfinx import mesh as msh
from mpi4py import MPI
from utils.transforms import channeled_flattening_transform, create_flattening_index_set

nx , ny = 32,32
domain = msh.create_unit_square(MPI.COMM_WORLD, nx, ny, msh.CellType.quadrilateral)
b = np.array([1,0]),
perturb_bool = False
x = mesh_to_x(
    mesh=domain,
    cell_ind_to_grid=create_cell_ind_to_grid(nx,ny),
    epsK=1e-8,
    bK=b,
    fK=1,
    cK=0,
    non_perturbed=perturb_bool,
    device='cpu',
    dtype=torch.float32
)


def x_to_mesh(x, dofmap):
    dofmap = torch.tensor(dofmap).flatten()
    x = x[:8,1:,:]
    x = x[:,:-1,:]
    x = x[:8,:,1:]
    x = x[:,:,:-1]
    #h,w = 
    cellvs = channeled_flattening_transform(flat_key=create_flattening_index_set(nx,ny,continuous_traversal=False),C=8)(x[:8])
    cellvs.T[0].reshape(-1,2)
    pts = []
    for vs in cellvs.T:
        pts.append(vs.reshape(-1,2))
    pts = torch.concat((pts))
    idx = torch.arange(torch.max(dofmap)+1)
    for j in range(len(idx)):
        idx[j] = torch.where(dofmap==j)[0][0]

    retval = torch.zeros((len(idx),3))
    retval[:,0:2] = pts[idx]
    return retval

retval = x_to_mesh(x, domain.geometry.dofmap)
torch.allclose(retval, torch.tensor(domain.geometry.x, dtype=retval.dtype))

True

In [66]:
x = x[:8,1:,:]
x = x[:,:-1,:]
x = x[:8,:,1:]
x = x[:,:,:-1]
x.shape[1]

32

In [ ]:
from dolfinx import mesh as msh
from mpi4py import MPI
from dolfinx import fem
from dolfinx import default_scalar_type
import ufl
import numpy as np
import torch

x = torch.load('data/example_3_1/training_set/inputs/x_1.pt')

def tensor_to_fem_data(x):
    bK = x[9:11, 1,1]
    if torch.allclose(bK, torch.tensor([ 1.0, 0.0], dtype = bK.dtype)):
        b = ufl.as_vector((fem.Constant(domain, default_scalar_type(1.0)),fem.Constant(domain, default_scalar_type(0.0))))
        dir = 0

    if torch.allclose(bK, torch.tensor([ -1.0, 0.0], dtype = bK.dtype)):
        b = ufl.as_vector((fem.Constant(domain, default_scalar_type(-1.0)),fem.Constant(domain, default_scalar_type(0.0))))
        dir = 0

    if torch.allclose(bK, torch.tensor([ 0.0, 1.0], dtype = bK.dtype)):
        b = ufl.as_vector((fem.Constant(domain, default_scalar_type(0.0)),fem.Constant(domain, default_scalar_type(1.0))))
        dir = 1

    if torch.allclose(bK, torch.tensor([ 0.0, -1.0], dtype = bK.dtype)):
        b = ufl.as_vector((fem.Constant(domain, default_scalar_type(0.0)),fem.Constant(domain, default_scalar_type(-1.0))))
        dir = 1


    ex_exp = x[dir]*(1-ufl.exp(-(1-x[dir])/eps))* (1 - (((ufl.exp(-(1-x[(dir+1)%2])/eps)  + ufl.exp(-(x[(dir+1)%2])/eps)))- ufl.exp(-(1)/eps))/(1-ufl.exp(-1/eps)))

    if torch.min(bK) < 0:
        ex_exp = (1-x[dir])*(1-ufl.exp(-(x[dir])/eps))* (1 - (((ufl.exp(-(1-x[(dir+1)%2])/eps)  + ufl.exp(-(x[(dir+1)%2])/eps)))- ufl.exp(-(1)/eps))/(1-ufl.exp(-1/eps)))


    
    uD = torch.concat((x[12,0,:],x[12,-1,:],x[12,:,0],x[12,:,-1]))
    if torch.allclose(max(uD), torch.tensor([ 0.0], dtype = bK.dtype)):
        ex_exp = x[dir]
    if np.min(bK) < 0:
        ex_exp = 1 - ex_exp

    #Ex 1
def pde_data1(domain):
    Wh = fem.functionspace(domain, ('P', 2))
    domain.topology.create_connectivity(domain.topology.dim - 1, domain.topology.dim)
    boundary_facets = msh.exterior_facet_indices(domain.topology)
    boundary_dofs = fem.locate_dofs_topological(Wh, domain.topology.dim-1, boundary_facets)
    eps = fem.Constant(domain, default_scalar_type(1e-8))
    c = fem.Constant(domain, default_scalar_type(0.0))
    f = fem.Constant(domain, default_scalar_type(1.0))
    bcs = [fem.dirichletbc(fem.Constant(domain, default_scalar_type(0.0)), boundary_dofs, Wh)]
    return Wh,eps,b,c,f,bcs,None

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=torch.float64)